The goal of this notebook is to compare two populations...one sampled from the NHANES dataset directly and one sampled from the Gaussian distributions which were in turn obtained from the NHANES dataset...hence the goal is to validate the Gaussian distributions we obtained...

First we read the df with the person information as obtained from the Gaussian distributions, then we create Person-objects, and then the population object. Finally we compare this population and the NHANESDirectSamplePopulation in baseline characteristics, characteristics over time, and cv events.

In [1]:
import os
import copy
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from scipy.stats import multivariate_normal

from microsim.population import (NHANESDirectSamplePopulation, 
                                 build_people_using_nhanes_for_sampling,
                                 microsimToNhanes, Population, get_nhanes_population)
from microsim.risk_model_repository import RiskModelRepository
from microsim.gender import NHANESGender
from microsim.smoking_status import SmokingStatus
from microsim.race_ethnicity import NHANESRaceEthnicity
from microsim.education import Education
from microsim.treatment import DefaultTreatmentsType,  TreatmentStrategiesType
from microsim.risk_factor import StaticRiskFactorsType, DynamicRiskFactorsType
from microsim.afib_model import AFibPrevalenceModel
from microsim.pvd_model import PVDPrevalenceModel
from microsim.outcome import Outcome, OutcomeType
from microsim.person import Person
from microsim.population_model_repository import PopulationModelRepository
from microsim.cohort_risk_model_repository import (CohortStaticRiskFactorModelRepository,
                                                   CohortDynamicRiskFactorModelRepository,
                                                   CohortDefaultTreatmentModelRepository)
from microsim.outcome_model_repository import OutcomeModelRepository

microsimDir = "/Users/deligkaris.1/OneDrive - The Ohio State University Wexner Medical Center/MICROSIM"
os.chdir(microsimDir+"/CODE/microsim")

dataDir = microsimDir+"/NOTEBOOKS/DATA"

popSize = 300000

In [2]:
categoricalVars = ["gender", "smokingStatus", "raceEthnicity", "statin",'education',
                  'alcoholPerWeek','anyPhysicalActivity','antiHypertensiveCount']
continuousVars = ['age', 'hdl', 'bmi', 'totChol', 'trig', 'a1c', 'ldl', 'waist', 'creatinine', 'sbp', 'dbp']

In [3]:
nhanesDf = pd.read_stata(microsimDir + "/CODE/microsim/microsim/data/fullyImputedDataset.dta")
nhanesDf = nhanesDf.rename(columns={"level_0":"name"})

In [4]:
df = pd.read_csv(dataDir+"/nhanes-as-gaussians-02.csv")
#df = pd.read_csv(dataDir+"/nhanes-persons-from-uniforms.csv")
df.head()

,name,gender,smokingStatus,raceEthnicity,statin,education,alcoholPerWeek,anyPhysicalActivity,antiHypertensiveCount,age,hdl,bmi,totChol,trig,a1c,ldl,waist,creatinine,sbp,dbp
0,41909,1,0,1,False,1,0,0,0.0,54,41.167309,25.694482,214.167769,210.549305,5.389349,148.330353,102.545480,0.688871,131.688014,78.255367
1,42688,1,0,1,False,1,0,0,0.0,51,46.384726,31.935565,164.732679,110.189826,5.286215,81.605992,112.511680,0.762213,137.004879,74.640230
2,43025,1,0,1,False,1,0,0,0.0,36,47.408049,28.315046,260.486802,126.640687,5.668748,177.731299,99.632114,0.857567,121.974670,90.400776
3,43390,1,0,1,False,1,0,0,0.0,40,45.379713,30.468159,215.360517,117.954243,5.686785,135.276734,103.727511,0.715434,113.273055,70.062711
4,44501,1,0,1,False,1,0,0,0.0,74,53.576906,22.736810,223.833927,63.617831,6.194046,170.082194,95.710354,0.696987,123.342909,81.288198


In [5]:
df["age"].describe()

count    5448.000000
mean       44.689611
std        16.418813
min        16.000000
25%        32.000000
50%        43.000000
75%        56.000000
max        93.000000
Name: age, dtype: float64

In [6]:
df.shape

(5448, 20)

In [7]:
#bring the original nhanes weights because these will help me do the sampling
df = df.merge(nhanesDf[["name","WTINT2YR"]], on="name", how="inner").copy()
df.shape

(5448, 21)

In [8]:
def build_person(x, initializationModelRepository):
    """Takes all Person-instance-related data via x and initializationModelRepository and organizes it,
       passes the organized data to the Person class and returns a Person instance."""
    
    rng = np.random.default_rng()
        
    name = x.name
    
    personStaticRiskFactors = {
                        StaticRiskFactorsType.RACE_ETHNICITY.value: NHANESRaceEthnicity(int(x.raceEthnicity)),
                        StaticRiskFactorsType.EDUCATION.value: Education(int(x.education)),
                        StaticRiskFactorsType.GENDER.value: NHANESGender(int(x.gender)),
                        StaticRiskFactorsType.SMOKING_STATUS.value: SmokingStatus(int(x.smokingStatus))}
        
    #use this to get the bounds imposed on the risk factors in a bit
    rfRepository = RiskModelRepository()
        
    #TO DO: find a way to include everything here, including the rfs that need initialization
    #the PVD model would be easy to implement, eg with an estimate_next_risk_for_patient_characteristics function
    #but the AFIB model would be more difficult because it relies on statsmodel_logistic_risk file
    #for now include None, in order to create the risk factor lists correctly at the Person instance
    personDynamicRiskFactors = dict()
    for rfd in DynamicRiskFactorsType:
        #if rfd==DynamicRiskFactorsType.ALCOHOL_PER_WEEK:
        #    personDynamicRiskFactors[rfd.value] = AlcoholCategory.get_category_for_consumption(rfRepository.apply_bounds(rfd.value, x[microsimToNhanes[rfd.value]]))
        #else:
        if (rfd!=DynamicRiskFactorsType.PVD) & (rfd!=DynamicRiskFactorsType.AFIB):
            personDynamicRiskFactors[rfd.value] = rfRepository.apply_bounds(rfd.value, x[rfd.value])
    personDynamicRiskFactors[DynamicRiskFactorsType.AFIB.value] = None
    personDynamicRiskFactors[DynamicRiskFactorsType.PVD.value] = None

    #Q: do we need otherLipid treatment? I am not bringing it to the Person objects for now.
    #A: it is ok to leave it out as we do not have a model to update this. It is also very rarely taking place in the population anyway.
    #also: used to have round(x.statin) but NHANES includes statin=2...
    personDefaultTreatments = {
                        DefaultTreatmentsType.STATIN.value: bool(x.statin),
                        #DefaultTreatmentsType.OTHER_LIPID_LOWERING_MEDICATION_COUNT.value: x.otherLipidLowering,
                        DefaultTreatmentsType.ANTI_HYPERTENSIVE_COUNT.value: x.antiHypertensiveCount}

    personTreatmentStrategies = dict(zip([strategy.value for strategy in TreatmentStrategiesType],
                                         #[None for strategy in range(len(TreatmentStrategiesType))]))
                                         [{"status": None} for strategy in range(len(TreatmentStrategiesType))]))

    personOutcomes = dict(zip([outcome for outcome in OutcomeType],
                                  [list() for outcome in range(len(OutcomeType))]))
    #add pre-simulation stroke outcomes
    #selfReportStrokeAge=x.selfReportStrokeAge
    #Q: we should not add the stroke outcome in case of "else"?
    #if selfReportStrokeAge is not None and selfReportStrokeAge > 1:
    #        selfReportStrokeAge = selfReportStrokeAge if selfReportStrokeAge <= x.age else x.age
    #        personOutcomes[OutcomeType.STROKE].append((selfReportStrokeAge, StrokeOutcome(False, None, None, None, priorToSim=True)))
    #add pre-simulation mi outcomes
    #selfReportMIAge=rng.integers(18, x.age) if x.selfReportMIAge == 99999 else x.selfReportMIAge
    #if selfReportMIAge is not None and selfReportMIAge > 1:
    #        selfReportMIAge = selfReportMIAge if selfReportMIAge <= x.age else x.age
    #        personOutcomes[OutcomeType.MI].append((selfReportMIAge, Outcome(OutcomeType.MI, False, priorToSim=True)))

    person = Person(name,
                   personStaticRiskFactors,
                   personDynamicRiskFactors,
                   personDefaultTreatments,
                   personTreatmentStrategies,
                   personOutcomes)

    #TO DO: find a way to initialize these rfs above with everything else
    person._pvd = [initializationModelRepository[DynamicRiskFactorsType.PVD].estimate_next_risk(person)]
    person._afib = [initializationModelRepository[DynamicRiskFactorsType.AFIB].estimate_next_risk(person)]
    return person


In [9]:
initializationModelRepository = {DynamicRiskFactorsType.AFIB: AFibPrevalenceModel(), 
                                     DynamicRiskFactorsType.PVD: PVDPrevalenceModel()}

In [10]:
dfSample = df.sample(popSize, weights=df.WTINT2YR, replace=True)

In [11]:
people = dfSample.apply(lambda x: build_person(x, initializationModelRepository), axis=1)

In [12]:
#sets the unique identifier for each Person instance
noneList = list(map(lambda person, i: setattr(person, "_index", i), people, range(len(people)))) 

In [13]:
popModelRepository = PopulationModelRepository(CohortDynamicRiskFactorModelRepository(),
                                                           CohortDefaultTreatmentModelRepository(),
                                                           OutcomeModelRepository(),
                                                           CohortStaticRiskFactorModelRepository()) 
pop = Population(people, popModelRepository)

In [14]:
pop._people.shape

(300000,)

In [15]:
#pop2 = get_nhanes_population(1999)
pop2 = NHANESDirectSamplePopulation(popSize, 1999)
pop2.advance(1)
#pop2.print_baseline_summary()

In [16]:
pop.print_baseline_summary_comparison(pop2)

                                 self                                             other
                                 min    0.25   med    0.75    max   mean    sd    min    0.25   med    0.75    max   mean    sd
                           age   18.0   33.0   43.0   55.0   93.0   44.7   15.4   18.0   30.0   41.0   55.0   85.0   43.8   17.3
                           sbp   77.4  112.8  122.1  133.8  221.1  124.0   16.7   72.7  109.3  119.3  132.0  266.0  122.7   19.1
                           dbp   37.9   65.7   72.3   78.8  121.9   72.4    9.8   40.0   65.3   72.0   79.3  132.0   72.4   11.2
                           a1c    3.7    5.0    5.2    5.6   10.1    5.3    0.6    2.5    5.0    5.2    5.5   15.1    5.4    0.9
                           hdl   18.0   42.4   51.1   60.4  114.1   52.1   13.4    8.0   40.0   48.0   59.0  151.0   50.8   15.6
                           ldl   38.2  106.6  127.2  148.6  238.7  128.2   30.8   28.0   98.0  120.0  144.0  354.0  123.0   36.7
          

In [17]:
popages = pop.get_age_of_all_years_in_sim()
popageCounts = pop.get_age_counts(popages)
popages2 = pop2.get_age_of_all_years_in_sim()
popageCount2s = pop2.get_age_counts(popages2)

print("age pop-count pop2-count %diff") 
for key in sorted(popageCounts):
    if key in popageCount2s:
        pcDiff =  (popageCounts[key] - popageCount2s[key])/popageCount2s[key]
        print(f"{key} {popageCounts[key]:>6.0f} {popageCount2s[key]:>10.0f} {pcDiff:>9.2f}")
    else:
        print(f"{key} {popageCounts[key]:>6.0f}")

age pop-count pop2-count %diff
18   4009       5340     -0.25
19   1922       6111     -0.69
20   2303       5799     -0.60
21   3825       6663     -0.43
22   3459       7261     -0.52
23   4294       5685     -0.24
24   4514       5244     -0.14
25   5728       5119      0.12
26   5602       6219     -0.10
27   4711       4618      0.02
28   6322       6414     -0.01
29   5819       6891     -0.16
30   6591       6006      0.10
31   6057       6170     -0.02
32   7623       7725     -0.01
33   5625       5978     -0.06
34   8558       6140      0.39
35   6220       7305     -0.15
36   7789       5975      0.30
37   8793       7733      0.14
38   5473       6274     -0.13
39   8597       8217      0.05
40   7634       6967      0.10
41   7147       6249      0.14
42   7949       6196      0.28
43   8476       5375      0.58
44   8963       6284      0.43
45   4982       6124     -0.19
46   7219       6353      0.14
47   6762       5324      0.27
48   5481       4596      0.19
49   618

In [18]:
#personsOut = list(filter( lambda x: ((x._sbp[0]>=72.) & (x._sbp[0]<=81.)) | ((x._sbp[0]>=207.) & (x._sbp[0]<=266.)) , pop2._people))
#len(personsOut)

In [19]:
#personsOutUnique = set(map( lambda x: x._name, personsOut) )
#len(personsOutUnique)

In [20]:
%%time
pop.advance(17, None, nWorkers=5)

CPU times: user 1min 39s, sys: 2min 24s, total: 4min 4s
Wall time: 29min 42s


In [21]:
%%time
pop2.advance(17, None, nWorkers=5)

CPU times: user 2min 40s, sys: 10min 51s, total: 13min 31s
Wall time: 46min 6s


In [22]:
pop.print_lastyear_summary_comparison(pop2)

                                 self                                             other
                                 min    0.25   med    0.75    max   mean    sd    min    0.25   med    0.75    max   mean    sd
                           age   34.0   47.0   56.0   67.0  103.0   57.3   12.9   35.0   45.0   55.0   65.0  102.0   56.0   13.6
                           sbp   89.5  123.3  131.3  140.3  209.1  132.3   13.2   79.8  120.3  128.6  138.5  276.1  130.1   15.0
                           dbp   47.7   74.2   81.7   89.4  128.7   82.0   10.8   41.4   73.7   81.8   90.0  152.5   81.9   11.6
                           a1c    4.4    5.5    5.8    6.1    8.5    5.8    0.4    4.0    5.5    5.7    6.1   10.7    5.8    0.6
                           hdl    5.4   39.5   50.7   63.6  118.1   51.9   17.9    5.4   35.0   47.1   62.3  170.6   49.7   21.1
                           ldl    8.1   95.9  113.2  129.5  212.0  112.2   25.3    8.1   94.2  114.9  133.3  286.2  112.9   30.2
          

In [23]:
%%time
pop.print_cv_standardized_rates()

standardized rates (per 100,000)    all        black   
                            mi      225.1      209.7
                        stroke      150.0      263.9
                         death      822.4     1029.2
                            cv      375.1      473.6
                         noncv      771.7      970.2
                      dementia      256.2      544.4
CPU times: user 6min 29s, sys: 9min 50s, total: 16min 20s
Wall time: 19min 39s


In [24]:
%%time
pop2.print_cv_standardized_rates()

standardized rates (per 100,000)    all        black   
                            mi      239.3      221.0
                        stroke      154.4      281.4
                         death      853.7      976.8
                            cv      393.7      502.6
                         noncv      800.6      907.5
                      dementia      275.4      563.4
CPU times: user 6min 45s, sys: 12min 15s, total: 19min
Wall time: 22min 49s
